In [ ]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from landuse_rb.classification import SpatialClassifier

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [ ]:
MERGE_DICT = {
    'recreation': 'sp_ag_rec',
    'special': 'sp_ag_rec',
    'agriculture': 'sp_ag_rec',
    'business': 'bus_res',
    'residential': 'bus_res',
    'industrial': 'industrial',
    'transport': None,

    'LandUse.RECREATION': 'sp_ag_rec',
    'LandUse.SPECIAL': 'sp_ag_rec',
    'LandUse.AGRICULTURE': 'sp_ag_rec',
    'LandUse.BUSINESS': 'bus_res',
    'LandUse.RESIDENTIAL': 'bus_res',
    'LandUse.INDUSTRIAL': 'industrial',
    'LandUse.TRANSPORT': None,    
}

In [ ]:
# Загрузка данных
with open('data/cleaned_blocks.pkl', 'rb') as f:
    gdf = pickle.load(f)
gdf.groupby('land_use').size().sort_values(ascending=False)
print(gdf.land_use.unique(), gdf.shape)

gdf = gdf[gdf.city == 'Saint Petersburg'] # Опционально для тестового запуска
train_gdf, test_gdf = train_test_split(gdf, test_size=0.15, random_state=42, stratify=gdf['land_use'])

In [ ]:
# 1. Инициализация и обучение
classifier = SpatialClassifier(buffer_distance=1000, k_neighbors=5)
classifier.fit(train_gdf, target_col='land_use_code')

In [ ]:
# 2. Предсказание на тестовых данных
test_predictions = classifier.predict(test_gdf)
test_probabilities = classifier.predict_proba(test_gdf)

In [ ]:
# 3. Сохранение обучающих данных
classifier.save_train_data('data/results/train_data.geojson')

# 4. Сохранение тестовых данных с предсказаниями
classifier.save_predictions_to_geojson(
    test_gdf, 
    test_predictions, 
    test_probabilities,
    'data/results/test_predictions.geojson'
)

# 5. Сохранение ошибок
classifier.save_mistakes(test_gdf, test_predictions, 'data/results/mistakes.geojson')